### Notebook for finding best precision settings for the first order perturbation theory computation

In [1]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys
import copy

font = {'size'   : 22}
matplotlib.rc("font", **font)
matplotlib.rc(["text.usetex", True])

sys.path.insert(0,'../source/')
import model 
import pert_first
import perturbation_old
import utils

In [2]:
x_here = np.array([8.0, 0.0, 0.0])
mnu = 0.1
Tnu = model.Tnu

rtol  = 1e-4

z_ini = 3.0
z_span = np.array([0, z_ini])

# base_rtols = [1e-4, 1e-4, 1e-3, 1e-2]
base_rtols = [1e-6, 1e-7, 1e-4, 1e-5]
rtol_scan = [1e-0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9] # Scan this list of tolerances for each of z, p, theta, phi

Gauss_Laguerre = 20
#Gauss_Laguerre = 0
#good_rtols = [1e-6, 1e-7, 1e-4, 1e-5]
good_rtols = np.array([1e-5, 0.0, 1e-4, 1e-3])
atols = np.array([1e-35, 1e-35, 1e-35, 1e-35]) # fixed

free_value = 1.06738178968e-10
asymptotic_value = 1.292761895e-10

### Timing a single run 

In [3]:
begin = time()
out = pert_first.integral(z_span, good_rtols, atols, x_here, mnu, Tnu, Gauss_Laguerre=Gauss_Laguerre)
print(f"After {time() - begin:.5} seconds: Found {1 + out/free_value}, compared to asymptotic value {asymptotic_value/free_value}.")

/Users/au566942/Documents/phd/projects/relic_density/KFT-Neutrinos/notebooks/../source/quadrature.py:40: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "GaussKronrod_adapt" failed type inference due to: non-precise type pyobject
During: typing of argument at /Users/au566942/Documents/phd/projects/relic_density/KFT-Neutrinos/notebooks/../source/quadrature.py (45)

File "../source/quadrature.py", line 45:
def GaussKronrod_adapt(f, a, b, rtol, abstol, f_args, is_indefinite=False):
    <source elided>
    # To integrate between 0 and infinity, set a=0, b=1 and is_indefinite=True on first call
    I, err = GaussKronrod_quad(f, a, b, is_indefinite, f_args)
    ^

  @jit(nopython=False)
/Users/au566942/miniforge3/envs/classpp/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "GaussKronrod_adapt" was compiled in object mode without forceobj=True.

File "../source/quadrature.py", line 41:
@jit(nopython=

After 3.8082 seconds: Found 1.212495186473708, compared to asymptotic value 1.2111522863694055.


After 1.8624 seconds: Found 1.212495186473708, compared to asymptotic value 1.2111522863694055.

2.5332 seconds:

In [4]:
tgsrtsergs

NameError: name 'tgsrtsergs' is not defined

After 3.4851 seconds: Found 1.2110028355063827, compared to asymptotic value 1.2111522863694055. (0.5)

**Gauss-Laguerre**: After 10.019 seconds: Found 1.211433956314203, compared to asymptotic value 1.2111522863694055. (41 points)

**Gauss-Laguerre**: After 21.99 seconds: Found 1.211640678753521, compared to asymptotic value 1.2111522863694055. (90 points)

**Gauss-Kronrod**: After 60.295 seconds: Found 1.211175599427068, compared to asymptotic value 1.2111522863694055.

After 3.8275 seconds: Found 1.2097454841015753, compared to asymptotic value 1.2111522863694055.

**Conclusions**: 

* Gauss-Kronrod/Gauss-Laguerre is not a big difference! Choose whichever you want! **Actually**, Gauss-Laguerre is better. Good config is ~40 GL points and the tolerances [1e-6, *, 1e-4, 1e-5]. Permille accuracy at ~10s.

* To acheive permille precision ~10 seconds evaluation time is needed.

### Adjusting the $z$ rel. tolerance

In [ ]:
import numba as nb

In [ ]:
KftArgs = namedtuple('KftArgs', 
                     field_names=['rtols', 'atols', 'f_evals', 'x_here', 'mnu', 'Tnu', 'G0_Gz', 
                                  'weight', 'Rs', 'p', 'theta', 'sintheta', 'costheta', 'Gauss_Laguerre'])

In [ ]:
nb.types.List(nb.float64[:, :])

(nb.types.NamedTuple([nb.float64[:], nb.float64[:], nb.int64, nb.float64[:], nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.float64, nb.int64], KftArgs))

@jit((nb.float64, nb.int64)
     (nb.float64[:], nb.float64[:], nb.float64[:], nb.float64[:], nb.float64, nb.float64, nb.int64), 
     nopython=True)
def integral(...):
    pass

In [ ]:
out_z = []

begin = time()
for rtol_z in rtol_scan:
    rtols = base_rtols.copy()
    rtols[0] = rtol_z
    out_z.append(pert_first.integral(z_span, rtols, atols, x_here, mnu, Tnu))
print(f"Finished after {time() - begin:.5} seconds.")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
ax.plot(rtol_scan, [out_z[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10)
ax.plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax.set(xscale='log', xlabel='rtol', ylabel='integral', title='Varying $z$ tolerance')
ax.invert_xaxis()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
ax.plot(rtol_scan, [out_z[idx][1] for idx in range(len(rtol_scan))], 'k.-', ms=10)
ax.set(xscale='log', yscale='log', xlabel='rtol', ylabel='# function evaluations')
ax.invert_xaxis()

### Adjusting $p$ tolerances

In [ ]:
out_p = []

begin = time()
for rtol_p in rtol_scan:
    rtols = base_rtols.copy()
    rtols[1] = rtol_p
    out_p.append(pert_first.integral(z_span, rtols, atols, x_here, mnu, Tnu))
print(f"Finished after {time() - begin:.5} seconds.")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
ax.plot(rtol_scan, [out_p[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10)
ax.plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax.set(xscale='log', xlabel='rtol', ylabel='integral', title='Varying $p$ tolerance')
ax.invert_xaxis()

### Adjusting $\theta$ tolerance

In [ ]:
out_th = []

begin = time()
for rtol_th in rtol_scan:
    rtols = base_rtols.copy()
    rtols[2] = rtol_th
    out_th.append(pert_first.integral(z_span, rtols, atols, x_here, mnu, Tnu))
print(f"Finished after {time() - begin:.5} seconds.")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
ax.plot(rtol_scan, [out_th[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10)
ax.plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax.set(xscale='log', xlabel='rtol', ylabel='integral', title='Varying $\theta$ tolerance')
ax.invert_xaxis()

### Adjusting $\phi$ tolerance

In [ ]:
out_phi= []

begin = time()
for rtol_phi in rtol_scan:
    rtols = base_rtols.copy()
    rtols[3] = rtol_phi
    out_phi.append(pert_first.integral(z_span, rtols, atols, x_here, mnu, Tnu))
print(f"Finished after {time() - begin:.5} seconds.")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
ax.plot(rtol_scan, [out_phi[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10)
ax.plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax.set(xscale='log', xlabel='rtol', ylabel='integral', title='Varying $\phi$ tolerance')
ax.invert_xaxis()

### Master plot

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=[10, 8])
xlims = [1e-10, 1e+1]
ax[0].plot(rtol_scan, [out_z[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10, label=r'varying $z$ tolerance')
#ax[0].plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax[0].set(xscale='log', xlabel='rtol', ylabel='integral', xlim=xlims, xticks=[])
ax[0].legend(frameon=False)
ax[0].invert_xaxis()

ax[1].plot(rtol_scan, [out_p[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10, label=r'varying $p$ tolerance')
#ax[1].plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax[1].set(xscale='log', xlabel='rtol', ylabel='integral', xlim=xlims, xticks=[])
ax[1].invert_xaxis()
ax[1].legend(frameon=False)

ax[2].plot(rtol_scan, [out_th[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10, label=r'varying $\theta$ tolerance')
#ax[2].plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax[2].set(xscale='log', xlabel='rtol', ylabel='integral', xlim=xlims, xticks=[])
ax[2].invert_xaxis()
ax[2].legend(frameon=False)

ax[3].plot(rtol_scan, [out_phi[idx][0] for idx in range(len(rtol_scan))], 'k.-', ms=10, label=r'varying $\phi$ tolerance')
#ax[3].plot(rtol_scan, np.ones(len(rtol_scan))*asymptotic_value, 'k--')
ax[3].set(xscale='log', xlabel='rtol', ylabel='integral', xlim=xlims)
ax[3].invert_xaxis()
ax[3].legend(frameon=False)
fig.subplots_adjust(hspace=0.0)